In [1]:
from transformers import T5Tokenizer,AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
train_data_path = "docs.txt"

In [3]:
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoModelWithLMHead

#データセットの設定
train_dataset = TextDataset(
    tokenizer = tokenizer,
    file_path = train_data_path,
    block_size = 128 #文章の長さを揃える必要がある
)

#データの入力に関する設定
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm= False
)

# 訓練に関する設定
training_args = TrainingArguments(
    output_dir="./result",  # 関連ファイルを保存するパス
    overwrite_output_dir=True,  # ファイルを上書きするかどうか
    num_train_epochs=3,  # エポック数
    per_device_train_batch_size=8,  # バッチサイズ
    logging_steps=100,  # 途中経過を表示する間隔
    save_steps=800  # モデルを保存する間隔
)

#トレーナーの設定
trainer = Trainer(
    model =model,
    args=training_args,
    data_collator = data_collator,
    train_dataset = train_dataset
)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [4]:
%%time
trainer.train()

Step,Training Loss
100,3.603800
200,3.108400


CPU times: user 26 s, sys: 2.26 s, total: 28.2 s
Wall time: 31.6 s


TrainOutput(global_step=210, training_loss=3.341394179207938, metrics={'train_runtime': 31.477, 'train_samples_per_second': 52.8, 'train_steps_per_second': 6.672, 'total_flos': 108566839296000.0, 'train_loss': 3.341394179207938, 'epoch': 3.0})

In [7]:
def getarate_sentences(seed_sentence):
    x = tokenizer.encode(seed_sentence, return_tensors="pt", 
    add_special_tokens=False)  # 入力
    x = x.cuda()  # GPU対応
    y = model.generate(x, #入力
                       min_length=50,  # 文章の最小長
                       max_length=100,  # 文章の最大長
                       do_sample=True,   # 次の単語を確率で選ぶ
                       top_k=50, # Top-Kサンプリング
                       top_p=0.95,  # Top-pサンプリング
                       temperature=1.2,  # 確率分布の調整
                       num_return_sequences=3,  # 生成する文章の数
                       pad_token_id=tokenizer.pad_token_id,  # パディングのトークンID
                       bos_token_id=tokenizer.bos_token_id,  # テキスト先頭のトークンID
                       eos_token_id=tokenizer.eos_token_id,  # テキスト終端のトークンID
                       # bad_word_ids=[[tokenizer.unk_token_id]]  # 生成が許可されないトークンID
                       )  
    generated_sentences = tokenizer.batch_decode(y, skip_special_tokens=True)  # 特殊トークンをスキップして文章に変換
    return generated_sentences

In [8]:
seed_sentence = "海洋開発とは"  
generated_sentences = getarate_sentences(seed_sentence)  # 生成された文章
for sentence in generated_sentences:
    print(sentence)

海洋開発とは「生物が、生物活動を通じて獲得可能な生態系機能を維持し、環境中で生き続けるシステムとしての基礎的機能」を育むことを提案するものであり、生態学の普遍的な知見を応用・再認識することにより生命の起源を明らかにする新たな一歩を開く重要な問いであり、さらに生物進化の科学の基盤である自然適応についての研究が推進されるべきであり、生態学的な生命発展の可能性を考える一助になると期待されます。さらに,微生物を含む生態学的進化がもたらす社会変動
海洋開発とは、気候変化や水の変化など様々な要因によって海洋環境が変化する現象のことを指し、2030年代に到来する温暖化の影響では気候変化が起きやすくなっていることを裏付ける貴重な証拠と言えます。近年、深海環境の深海底で、海底堆積物が二酸化炭素を吸収しながら海底が冷却することにより溶け出す過程が見つかっており、それが温暖化の進行に深く関与していることが示唆されています。このような背景から、海洋開発がもたらすさまざまな課題()に対して
海洋開発とは、海洋の気候の力学的な構成の変化として捉えられてきた温暖化と水循環の変化に起因して、生物活動の促進を促していることが知られています(図4, 6)。つまり、気候変動が海洋生態系の応答性を向上させ、生態系をどのように回復していくかということが、海洋生態系にとっては非常に重要であることを意味し、今後の水循環・海洋システムの変動にとって重要な課題であるとの認識から、本研究では深海海洋調査船「しんかい6500」「しんかい
